In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

import radiative_transfer as rt
import canary_helper_functions as chf

In [ ]:
# Constants
m_p = 1.67*10**-24 # Proton mass [g]
k_b = 1.38*10**-16 # Boltzmann constant [J/k]
c = 3*10**5        # Speed of light [km/s]

## General inputs

In [ ]:
#===============================================================================
# Fill in the input- and lineparameters here

N_mc = 10**5 # Number of MC photon packets to simulate. Default 10**6
N_bin = 400  # Number of wavelength bins for the emergent spectrum

T = 5000 # Temperature in the relevant zone. Only used for thermal reshuffling and exact choice of T should thus not matter much.
v_lim = 10 # photospheric radius, [kms]. Default: 10
wlmin = 8200 # The minimum wl [Å] of the considered grid. Default 8200 
wlmax = 9100 # The maximum wl [Å] of the considered grid. Default 9100 
line_shape = 'parabola' # can be 'parabola' or 'gaussian'. Default 'parabola'
save_profiles = True # Do you want to save the profiles that you create?

## Line selection

In the cell below, one can define which lines should be included in the simulation. You can add your own lines of interest in any way you want. In the Barmentloo+25 paper, the present lines were:
1 O I 8446,
2 Ca II 8498,
3 Ca II 8542,
4 Ca II 8662,
5 C I 8727,
6 Mg I 8806,

The first four were assumed to be optically thick at all times. On the contrary, C I 8727 was assumed to always be optically thin. Finally, Mg I 8806 was allowed to vary its optical depth from 0 to 2.

In [ ]:
# Lines are defined as lists, where each list element means the following:
# el 0: rest wavelength  
# el 1: transition strength A [s^-1]
# el 2: is the line optically thick? (This is overruled if a 'tau_[ion]_list' exists)

line_OI_8446 = [8446, 9.66*10**7, True]
line_CaII_8498 = [8498, 1.11*10**6, True]
line_CaII_8542 = [8542, 9.9*10**6, True]
line_CaII_8662 = [8662, 1.06*10**7, True]
line_CI_8727 = [8727, 0.6, False]
line_MgI_8806 = [8806, 1.27*10**7, True]

#CARE these must be sorted by increasing wl!
lines_list = [line_OI_8446, line_CaII_8498, line_CaII_8542, line_CaII_8662, line_CI_8727, line_MgI_8806] 
N_lines = len(lines_list)

Here, one can enter which combinations of line velocity and line luminosity one would like to test. 

Line velocity is a single velocity for all lines. The line luminosity is a number that is defined in a relative way. Its only meaning is thus relative to the sum of all line luminosities. For each possible combination of the lists below, the code will generate a line profile and store it in the 'profile_library' folder. The file name of this profile is defined in 'canary_helper_functions.py'.

In [ ]:
# The free parameters
v_outs = np.arange(3000, 7001, 500) # The velocity above which photons escape (effectively the line width)
P_CaII_list = [5] # CARE In the 2025 paper, this was set to 5, so all lineprofile names from the paper are scaled relative to that. 
P_CI_list = [0, 0.25, 0.5, 0.75, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20]
P_OI_list = [0]
P_MgI_list = [0]
tau_MgI_list = [0, 2]

## Code start

The code starts working below, you should not have to change anything here!

In [ ]:
# Initialize arrays that will be used in the Monte Carlo routine
tau0vec = np.zeros(N_lines)
wllines = np.zeros(N_lines)
destrprob = np.zeros(N_lines)

for i in range(N_lines):
    if lines_list[i][2] == True:
        tau0vec[i] = 10**10 # Optically thick
    else:
        tau0vec[i] = 10**-10 # Optically thin
        
    wllines[i] = lines_list[i][0]
    destrprob[i] = 0 # Default = 0. This means photons can only scatter, not be absorbed
    
    
binarr = np.linspace(wlmin,wlmax,N_bin)

In [ ]:
if 'profile_library' not in os.listdir(os.getcwd()):
    os.mkdir('profile_library')
    print("The folder 'profile_library' was not found, so it has been created.")


# The main loop
for q in range(len(v_outs)):
    hist_list, histscatt_list = [], []
    
    for s in range(len(P_CaII_list)):
        for t in range(len(P_CI_list)):
            for u in range(len(P_OI_list)):
                for v in range(len(P_MgI_list)):
                    for w in range(len(tau_MgI_list)):
                        
    
                        # Set the input parameters for this line profile
                        v_out = v_outs[q]
                        rin = v_lim/v_out

                        P_CaII, P_CI, P_OI, P_MgI = P_CaII_list[s], P_CI_list[t], P_OI_list[u], P_MgI_list[v],
                        el_probabilities = np.array([P_CaII, P_CI, P_OI, P_MgI])
                        
                        tau_MgI = tau_MgI_list[w]
                        tau0vec[np.where(wllines == 8806)[0][0]] = tau_MgI 

                        # Determine which lines are optically thick
                        scattering_lines = np.array([True, False, False, False])
                        for b in range(len(lines_list)):
                            if lines_list[b][0] == 8446:
                                scattering_lines[2] = True
                            elif lines_list[b][0] == 8806 and tau_MgI != 0:
                                scattering_lines[3] = True
                                
                        # Check first if this profile already exists ; if yes, continue
                        line_profile_name = chf.extract_line_profile_fname(el_probabilities, scattering_lines, tau_MgI, v_out, line_shape)
                        
                        already_created_names = os.listdir(os.getcwd() + '/profile_library/')
                        
                        if line_profile_name in already_created_names:
                            print('This profile already exists! Continuing with the next one ...')
                            continue

                        
                        print('Below we create a line profile for the following input parameters:')
                        print('v_out, [P_CaII, P_CI, P_OI, P_MgI], tau_MgI')
                        print(v_out, ', ', el_probabilities, ', ', tau_MgI)

                        n_destroyed, n_scatterdeath = 0, 0
                        hist = np.zeros(N_bin)
                        histscatt = np.zeros(N_bin)

                        for i in tqdm(range(N_mc)):

                            # Initialise photon packet state
                            scattered = False
                            escaped = False
                            destroyed = False
                            n_scatters = 0
                            printit = False

                            # Get the starting wavelength, angle of the packet
                            wl, mu, r = rt.draw_random_photon(wllines, line_shape, el_probabilities, wlmin, wlmax, rin)
                            sin_theta = np.sqrt(1 - mu**2)


                            while (escaped == False and destroyed == False): # While the packet is alive

                                # Compute distance to outer shell edge:
                                distouter = -r*mu + np.sqrt((r*mu)**2 - (r**2 - 1**2))

                                # Compute distance to inner shell edge:
                                if (mu < 0 and r*sin_theta < rin):
                                    distinner = -r*mu - np.sqrt((r*mu)**2 - (r**2 - rin**2))
                                else:
                                    distinner = 2.

                                # Draw random optical depth of this photon
                                xi = random.random()
                                tau_thisphoton = -np.log(1-xi) # A number between 0 and +infinity

                                # Now calculate the distance to the next scattering line

                                redward_indices = (wllines > wl) # Get all lines redward of here
                                if (any(redward_indices)):
                                    ind_closest_line = np.where(wllines == wllines[redward_indices][0])[0][0]
                                    wl_closest = wllines[ind_closest_line]
                                    distscatt = ( (wl_closest - wl) / wl ) * (c/v_out) # Note: this is normalised to the outer radius at v_out (max(distscatt) = 2)
                                else: 
                                    distscatt = 10 #No redward line remaining, so no interaction

                                # Update sintheta
                                sin_theta = np.sqrt(1.-mu**2)


                                if (distscatt < distouter and distscatt < distinner): # If distance to interaction is smaller than to both shell edges..
                                    n_scatters += 1
                                    wl, r, mu, sin_theta, destroyed, scattered = rt.check_interaction(distscatt, wl, mu, r, sin_theta, v_out, T, tau0vec, destrprob, ind_closest_line, destroyed, scattered)

                                elif (distouter < distinner): # Next event is hitting outer shell edge..:  
                                    escaped, hist, histscatt = rt.do_escape(distouter, wl, r, mu, sin_theta, v_out, scattered, hist, histscatt, binarr)

                                else: # Next event is hitting inner shell edge
                                    destroyed = True #Does it die?

                                    n_destroyed += 1

                                #=====================================================================================
                                #As destrprob = 0, photons can get trapped around line wavelength, scattering forever. This if-statement avoids that
                                if n_scatters > 50:
                                    destroyed = True

                                    n_scatterdeath += 1

                        hist = chf.remove_continuum_flux(binarr, hist)[0] # Removes a flat continuum and normalises to the profile maximum
                        histscatt = chf.remove_continuum_flux(binarr, histscatt)[0]

                        hist_list.append(hist)
                        histscatt_list.append(histscatt)

                        if save_profiles == True:
                        
                            chf.save_profile(binarr, hist, el_probabilities, scattering_lines, v_out, tau_MgI, line_shape)
                    
                        chf.plot_result(hist_list, histscatt_list, binarr, n_destroyed, n_scatterdeath)